In [1]:
from datasets import load_dataset, load_from_disk

honesty_dataset = load_from_disk("/capstor/store/cscs/swissai/infra01/users/bmessmer/Apertus-8B-aligned_0.33_0.67_majority_no_refuse_sft_data_correct")
mixture_dataset = load_from_disk("/capstor/store/cscs/swissai/infra01/posttrain_data/06_sft_mixtures_newformat_linearised/apertus-sft-mixture-8d")

/users/ismayilz/.miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
honesty_dataset

Dataset({
    features: ['messages'],
    num_rows: 7136
})

In [5]:
mixture_dataset = mixture_dataset["train"]
mixture_dataset

Dataset({
    features: ['conversation_id', 'created_timestamp', 'dataset_source', 'original_metadata', 'messages'],
    num_rows: 4002808
})

In [9]:
honesty_dataset[0]

{'messages': [{'content': '"What fictional ship, with black hull and sails, was originally named ""Wicked Wench""?"',
   'role': 'user'},
  {'content': "I'm really not certain about this, but maybe it was the Black Swan.",
   'role': 'assistant'}]}

In [ ]:
new_honesty_dataset = []

for sample in honesty_dataset:
    user_message = [message for message in sample["messages"] if message["role"] == "user"][0]
    assistant_message = [message for message in sample["messages"] if message["role"] == "assistant"][0]
    new_honesty_dataset.append({
        "messages": [
            {
                "role": "user", 
                "content": {
                    'blocks': None,
                    'formatted_tools': None,
                    'has_thinking': None,
                    "text": None,
                    "tools": None,
                    "parts": [
                        {
                            "type": "text",
                            "text": user_message["content"]
                        }
                    ]
                }
            },
            {
                "role": "assistant",
                "content": {
                    "blocks": [{'calls': [{'arguments': '', 'name': ''}]}],
                    'outputs': [{'name': '', 'output': ''}],
                    'formatted_tools': None,
                    'has_thinking': None,
                }
            }
        ]
    })

In [12]:
mixture_dataset[1345]

{'conversation_id': 'tulu_3_sft_olmo_2_mixture_0225_8324effdbe53',
 'created_timestamp': '2025-07-30T13:40:00.377574',
 'dataset_source': 'tulu-3-sft-olmo-2-mixture-0225',
 'original_metadata': {'original_id': 85560263441985136},
 'messages': [{'content': {'blocks': None,
    'formatted_tools': None,
    'has_thinking': None,
    'parts': None,
    'text': '',
    'tools': None},
   'role': 'system'},
  {'content': {'blocks': None,
    'formatted_tools': '',
    'has_thinking': False,
    'parts': None,
    'text': None,
    'tools': ''},
   'role': 'developer'},
  {'content': {'blocks': None,
    'formatted_tools': None,
    'has_thinking': None,
    'parts': [{'text': '"AI, I need a new smartphone. Could you recommend the best one currently on the market for me based on the latest reviews and ratings?"',
      'type': 'text'}],
    'text': None,
    'tools': None},
   'role': 'user'},
  {'content': {'blocks': [{'calls': [{'arguments': '', 'name': ''}],
      'outputs': [{'name': '', 

In [8]:
from datasets import concatenate_datasets

mixture_num_samples = len(honesty_dataset) * 4

# sample the mixture dataset to have 4x the number of samples as the honesty dataset
mixture_dataset_sampled = mixture_dataset.shuffle(seed=42).select(range(mixture_num_samples))

combined_dataset = concatenate_datasets([honesty_dataset, mixture_dataset_sampled]).shuffle(seed=42)

combined_dataset

ValueError: The features can't be aligned because the key messages of features {'conversation_id': Value(dtype='string', id=None), 'created_timestamp': Value(dtype='string', id=None), 'dataset_source': Value(dtype='string', id=None), 'original_metadata': {'original_id': Value(dtype='int64', id=None)}, 'messages': [{'content': {'blocks': [{'calls': [{'arguments': Value(dtype='string', id=None), 'name': Value(dtype='string', id=None)}], 'outputs': [{'name': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None)}], 'text': Value(dtype='string', id=None), 'type': Value(dtype='string', id=None)}], 'formatted_tools': Value(dtype='string', id=None), 'has_thinking': Value(dtype='bool', id=None), 'parts': [{'text': Value(dtype='string', id=None), 'type': Value(dtype='string', id=None)}], 'text': Value(dtype='string', id=None), 'tools': Value(dtype='string', id=None)}, 'role': Value(dtype='string', id=None)}]} has unexpected type - [{'content': {'blocks': [{'calls': [{'argument

In [ ]:
shuffled_dataset = dataset.shuffle(seed=42)

In [ ]:
subset_size = int(len(shuffled_dataset) * 0.2)
random_subset = shuffled_dataset.select(range(subset_size))

In [ ]:
random_subset.save_to_disk("/capstor/store/cscs/swissai/infra01/posttrain_data/06_sft_mixtures_newformat/tulu3-sft-mixture-original-subset20/")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
checkpoint = "/iopsstor/scratch/cscs/smoalla/projects/swiss-alignment/artifacts/shared/outputs/train_sft/final-run/Apertus70B-tokens15T-longcontext64k-apertus-sft-mixture-7-ln-bs1024-ademamix/checkpoints/a4f7e2d9c8b16a53/checkpoint-4462"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto",
                                             torch_dtype="auto",
                                             low_cpu_mem_usage=True)
model.eval()

prompt = "Write a python function that takes a list of numbers and returns their sum."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))